# Dive-into-DL-LibTorch

`---《动手学深度学习》LibTorch（pytorch的c++库）版本`

近年来，在服务端和边缘侧部署深度学习推理引擎时，人们逐渐从python过渡到c++编程，这是因为c/c++程序在执行效率等方面的优势使然，正如PyTorch官方教程（[USING THE PYTORCH C++ FRONTEND](https://pytorch.org/tutorials/advanced/cpp_frontend.html)）所述，相对于python，c++的优势有：
* Low Latency Systems
* Highly Multithreaded Environments
* Existing C++ Codebases

特别是多线程并发执行、程序执行效率、多媒体输入输出及编解码方面，c++还是非常灵活并好用的，当然了，其学习门槛也相对高一些。
本项目正是为了解和熟悉libtorch api的使用以及掌握相关编程技巧而产生的，希望能对您有所帮助。

# Chapter 00 - 配置可执行c++程序的jupyter notebook

jupyter notebook作为一款时下流行的网页版的交互式计算工具，非常适合演示深度学习相关算法编程过程。以往jupyter notebook多支持python等解释性语言，但是借助于“**xeus-cling**”，jupyter notebook也能执行C++程序了！

## 安装xeus-cling

请先在电脑上准备好jupyter notebook，如果没有，推荐安装Anaconda 3， 具体操作请参考anaconda官网安装教程。

执行下面语句，创建一个新的conda环境并安装xeus-cling：

```sh
conda create -n my_cpp_env python=3    
conda activate my_cpp_env    
conda install -c conda-forge xeus-cling    
```

关于xeus-cling的使用，推荐看<span class="mark">[官方教程](https://xeus-cling.readthedocs.io/)</span>。


## 编译安装libtorch

虽然pytorch官网有现成编译好的版本，但是考虑的系统环境的兼容问题，还是推荐自行编译libtorch，具体方法请参考[libtorch (C++-only)](https://github.com/pytorch/pytorch/blob/master/docs/libtorch.rst)，我使用的是“**Building libtorch using CMake**”方式，另外，git clone完pytorch代码库后，要执行一下"git submodule sync"和"git submodule update --init --recursive"两条指令来同步sub module。
```sh
git clone -b master --recurse-submodule https://github.com/pytorch/pytorch.git
cd pytorch
git submodule sync
git submodule update --init --recursive
mkdir pytorch-build
cd pytorch-build
cmake -DBUILD_SHARED_LIBS:BOOL=ON -DCMAKE_BUILD_TYPE:STRING=Release -DPYTHON_EXECUTABLE:PATH=`which python3` -DCMAKE_INSTALL_PREFIX:PATH=../pytorch-install ../pytorch
cmake --build . --target install
```

下面我们就体验一下在jupyter notebook中编写c++程序。
 
> 注：笔者的环境是ubuntu 18.04 LTS + libtorch v1.7.1版本

## 配置第三方库以及头文件的路径

因为没有配置cmake或者makefile的地方，因此，对于系统标准路径外的第三方库，我们需要手动导入。我分别在Ubuntu(18.04)和MacOS(High Sierra, 10.13.6)上进行了配置，因为cling的bug，我们在MacOS上编译自己的代码时，会遇到类似下面的报错：

```
cling::DynamicLibraryManager::loadLibrary(): dlopen(/Volumes/code/github/libtorch/pytorch-install/lib/libtorch.dylib, 9): 
* Library not loaded: @rpath/libtorch_cpu.dylib
* Referenced from: /Volumes/code/github/libtorch/pytorch-install/lib/libtorch.dylib
* Reason: image not found
```

我这边尝试过网络上的几种方案，包括[cling issue 370](https://github.com/root-project/cling/issues/370)中提到的手动编译cling的办法以及[Setting-Up-Xeus-Cling-Libtorch-OpenCV](https://krshrimali.github.io/Setting-Up-Xeus-Cling-Libtorch-OpenCV/)一文中提到的修改cmake文件中`set(TORCH_CXX_FLAGS "-D_GLIBCXX_USE_CXX11_ABI=")`为`set(TORCH_CXX_FLAGS "-D_GLIBCXX_USE_CXX11_ABI=1")`的办法，均不生效。最后，还是通过[xeus-cling issue 161](https://github.com/jupyter-xeus/xeus-cling/issues/161)的workaround来解决这个编译问题的，即在你的每个jupyter notebook文件前面先运行161 issue中给的workaround代码，因代码过长，此处不再引用，为方便起见，也可以把这段workaround代码放在头文件中，每次启动时include该头文件即可。代码具体内容请参见`macos_cling_workaround.hpp`文件。




In [1]:
#pragma cling add_library_path("/usr/local/lib")
#pragma cling add_include_path("/usr/local/include")

#if defined(__linux__)

#pragma cling add_library_path("/home/ubuntu/liuhang/libtorch/pytorch-install/lib")
#pragma cling add_include_path("/home/ubuntu/liuhang/libtorch/pytorch-install/include")
#pragma cling add_include_path("/home/ubuntu/liuhang/libtorch/pytorch-install/include/torch/csrc/api/include")

#pragma cling load("libc10_cuda.so")
#pragma cling load("libc10.so")
#pragma cling load("libtorch.so")

#elif __APPLE__

#pragma cling add_library_path("/Users/liuhang/codes/github/libtorch/release/lib")
#pragma cling add_include_path("/Users/liuhang/codes/github/libtorch/release/include")
#pragma cling add_include_path("/Users/liuhang/codes/github/libtorch/release/include/torch/csrc/api/include")

#pragma cling load("libtorch.dylib")

#endif

> 显然，每次在jupyter notebook文件中显式的声明头文件/库文件路径，以及显式的加载动态库，确实是比较麻烦，因此，我建议还是把上述代码统一放到一个头文件中，这样会相对简洁一些。如下面例子所示，我将上述的声明都放在"load_libtorch.hpp"中。如果后续再引入opencv之类的第三方库，管理也会更便利一些。

导入相关头文件，c/c++都可以。

In [1]:
#include "stdio.h"
#include "stdlib.h"
#include <iostream>

#include "../macos_cling_workaround.hpp"
#include "../load_libtorch.hpp"
#include <torch/torch.h>

打印你的“Hello world”！

In [2]:
printf("Hello, jupyter-notebook!\r\n")

Hello, jupyter-notebook!


26

In [3]:
std::cout << "This is a c++ programme in jupyter notebook!" << std::endl;

This is a c++ programme in jupyter notebook!


测试一下libtorch，创建一个tensor：

In [4]:
torch::Tensor tensor = torch::eye(5);
std::cout << tensor << std::endl;
    

 1  0  0  0  0
 0  1  0  0  0
 0  0  1  0  0
 0  0  0  1  0
 0  0  0  0  1
[ CPUFloatType{5,5} ]


@0x102921c30